In [1]:
import pandas as pd

# 1. Cargar el archivo
ruta_archivo = 'base_transacciones_final.csv'
df = pd.read_csv(ruta_archivo, low_memory=False)

resumen_na = (
    df.isna()
      .sum()
      .reset_index()
      .rename(columns={'index': 'columna', 0: 'valores_faltantes'})
)

# 3. Calcular porcentaje de faltantes
resumen_na['porcentaje'] = (resumen_na['valores_faltantes'] / len(df) * 100).round(2)

# 4. Mostrar resultado
print("Resumen de valores faltantes por columna:")
print(resumen_na)


Resumen de valores faltantes por columna:
         columna  valores_faltantes  porcentaje
0             id                  0        0.00
1          fecha                  0        0.00
2       comercio                  0        0.00
3  giro_comercio               5588        1.61
4     tipo_venta                  0        0.00
5          monto                  0        0.00


In [2]:
missing_by_comercio = (
    df[df['giro_comercio'].isna()]
    .groupby('comercio')
    .size()
    .reset_index(name='missing_count')
    .sort_values(by='missing_count', ascending=False)
)

print(missing_by_comercio.head(10))  # top 10

#Too much data for missing so replacing the value


                 comercio  missing_count
3                   APPLE           4822
4                 AUDIBLE            169
8              GOOGLE ONE            142
11    PLAYSTATION NETWORK            132
2            AMAZON PRIME            104
10  GOOGLE YOUTUBEPREMIUM             83
6                  GOOGLE             68
9          GOOGLE YOUTUBE             41
1                  AMAZON             10
7    GOOGLE AMAZON MOBILE             10


In [3]:
map_comercio_giro = (
    df.dropna(subset=['giro_comercio'])
      .groupby('comercio')['giro_comercio']
      .agg(lambda x: x.mode().iloc[0])  # "el más repetido"
)
print(map_comercio_giro)

comercio
7 ELEVEN                         TIENDAS DE CONVENIENCIA, MINISUPER
7ELEVEN                          TIENDAS DE CONVENIENCIA, MINISUPER
ADOBE                SERV PROC DATOS - DISENO Y PROGRAM DE SISTEMAS
ALIEXPRESS                                     TIENDA DEPARTAMENTAL
ALLIANZ MEXICO            ASEGURADORAS - VENTAS DE SEGUROS, POLIZAS
                                        ...                        
UNDOSTRES         SERVICIOS DE TELECOMUNICACION, LLAMADAS LOC Y ...
URBANI                      SERVICIOS DE TRANSPORTE -NO CLASIFICADO
VIX                                                       AGREGADOR
WAL-MART                                       TIENDA DEPARTAMENTAL
WALMART                               AUTOSERVICIOS Y SUPERMERCADOS
Name: giro_comercio, Length: 96, dtype: object


In [4]:
missing_by_comercio = (
    df[df['giro_comercio'].isna()]
    .groupby('comercio')
    .size()
    .reset_index(name='missing_count')
    .sort_values(by='missing_count', ascending=False)
)

print(missing_by_comercio.head(10))  # top 10

#Too much data for missing so replacing the value

                 comercio  missing_count
3                   APPLE           4822
4                 AUDIBLE            169
8              GOOGLE ONE            142
11    PLAYSTATION NETWORK            132
2            AMAZON PRIME            104
10  GOOGLE YOUTUBEPREMIUM             83
6                  GOOGLE             68
9          GOOGLE YOUTUBE             41
1                  AMAZON             10
7    GOOGLE AMAZON MOBILE             10


In [5]:
df['giro_comercio'].dropna().unique()


array(['COMERCIOS ELECTRONICOS (VTAS POR INTERNET)',
       'SERVICIOS EMPRESARIALES - NO CLASIFICADOS',
       'SERV GUBERNAM, AGUA, ELECTRICIDAD,PREDIAL,TENENCIA',
       'TIENDAS DE CONVENIENCIA, MINISUPER', 'DROGUERIAS, (FARMACIAS)',
       'AUTOSERVICIOS Y SUPERMERCADOS',
       'TRASLADOS - FLETES, (SERVICIOS DE MENSAJERIA)',
       'LIMOSINAS, (TAXIS)', 'TIENDA DEPARTAMENTAL', 'AGREGADOR',
       'CLUBES DE MAYORISTA',
       'ESTACIONES DE SERVICIO, GASOLINERIAS TODO TRANSPOR',
       'COMIDA RAPIDA', 'RESTAURANTES',
       'MERCADO DIRECTO - VENTAS POR CATALOGO',
       'SERVICIOS DE TELECOMUNICACION, LLAMADAS LOC Y LARG',
       'MERCADEO DIRECTO - NO CLASIFICADO',
       'COMPUTADORAS EQUIPO PERIF DE COMPUTADORAS SOFTWARE', '4121',
       'PAGO DE SERVICIO POR TELEVISION CABLE/POR EVENTO',
       'SERVICIOS PUBLICITARIOS', 'CINES, TEATROS',
       'SERVICIOS PROFESIONALES - NO CLASIFICADOS',
       'MERCADEO DIRECTO - COMERCIANTES DE TELEMERCADEO',
       'RENTA DE VIDEOS  (

In [6]:
#Replace the only unknown giro to the best fit, audible is MEDIOS DE COMUNICACION DIGITAL LIBROS PELICULAS M
audible_giro = 'MEDIOS DE COMUNICACION DIGITAL LIBROS PELICULAS MU'
df.loc[df['comercio'] == 'AUDIBLE', 'giro_comercio'] = audible_giro

In [7]:
import numpy as np

# 1) Compute the mode for each comercio
mode_per_comercio = (
    df
    .groupby('comercio')['giro_comercio']
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
)

# 2) Fill the NaNs in giro_comercio by mapping each comercio to its mode
df['giro_comercio'] = df['giro_comercio'].fillna(
    df['comercio'].map(mode_per_comercio)
)

# 3) (Optional) Check how many still remain
print("remaining NaNs:", df['giro_comercio'].isna().sum())


remaining NaNs: 0


In [8]:
import re
import unicodedata

# 1) A small helper to:
#    – lowercase
#    – strip accents (so “á” → “a”)
#    – remove ANY non-alphanumeric
def normalize_name(s):
    # make sure it’s a str
    s = str(s).lower()
    # strip accents
    s = unicodedata.normalize('NFKD', s).encode('ascii', 'ignore').decode('utf-8')
    # drop everything except a–z and 0–9
    return re.sub(r'[^a-z0-9]', '', s)

# 2) Apply to create a new column
df['comercio_norm'] = df['comercio'].apply(normalize_name)

# 3) Now you can inspect uniques on the normalized names:
print(df['comercio_norm'].unique())

# 4) (Optional) If you want to see the original → normalized mapping:
mapping = (
    df[['comercio', 'comercio_norm']]
      .drop_duplicates()
      .sort_values('comercio_norm')
)
print(mapping.to_string(index=False))


['amazon' 'rappi' 'cfe' 'oxxo' 'farmaciasdelahorro' '7eleven' 'soriana'
 'heb' 'liverpool' 'spotify' 'supercenter' 'costco' 'oxxogas' 'uber'
 'starbucks' 'ubereats' 'servaguadren' 'amazonprime' 'undostres'
 'mercadopago' 'rappipro' 'walmart' 'telcel' 'telmex' 'crunchyroll'
 'farmaciasguadalajara' 'izzi' 'cinepolis' 'disneyplus' 'bait' 'nayax'
 'carlsjr' 'apple' 'totalplay' 'samsclub' 'netflix' 'melimas' 'microsoft'
 'max' 'tulotero' 'bae' 'didirides' 'googleyoutubepremium' 'allianzmexico'
 'itunes' 'smart' 'alsuper' 'ubrpagosmex' 'parco' 'facebook' 'temu'
 'chedraui' 'coppel' 'farmaciassimilares' 'google' 'miatt' 'superama'
 'rotoplas' 'totalpass' 'aliexpress' 'caliente' 'didi' 'didifood'
 'cashiecommerce' 'googleone' 'aplazo' 'aplaz' 'softrappi' 'openai' 'vix'
 'shein' 'costcogas' 'att' 'smartfit' 'googleyoutube' 'cableycomun'
 'kueskipay' 'canva' 'urbani' 'bet365' 'sears' 'rentamovistar'
 'playstationnetwork' 'megacable' 'telefonica' 'adobe' 'metrobus'
 'googleamazonmobile' 'audible'

In [46]:
df['comercio_norm'] = df['comercio'].apply(normalize_name)

# 2) drop the raw one and rename the normalized
df = (
    df
    .drop(columns=['comercio'])
    .rename(columns={'comercio_norm':'comercio'})
)

In [47]:
CATEGORIES = {
    'domiciliado': {
        'cfe','telmex','izzi','totalplay','totalpass',
        'rentamovistar','telefonica','megacable','att','adobe',
        'itunes','googleone'
    },
    'comida': {
        'rappi','ubereats','didifood','didirides',
        'starbucks','carlsjr', 'softrappi'
    },
    'entretenimiento': {
        'netflix','spotify','disneyplus','crunchyroll',
        'vix','max','playstationnetwork','amazonprime',
        'googleyoutubepremium','googleyoutube','roku','bait','sears','audible',
        'cinepolis','bet365','facebook'
    },
    'retail_online': {
        'amazon','aliexpress','mercadopago','shein','temu','googleamazonmobile'
    },
    'retail_fisico': {
        'walmart','soriana','heb','chedraui','costco','alsuper',
        'samsclub','farmaciasdelahorro','farmaciasguadalajara','superama',
        'oxxo','liverpool','supercenter','coppel','farmaciassimilares','rotoplas'
    },
    'transporte': {
        'uber','didi','didirides','7eleven','metrobus','parco'
    },
    'telecom': {
        'telcel','smart','kueskipay','iri','ubi','smartfit','cableycomun'
    },
    'finanzas': {
        'paypal','kueskipay','mercadopago','tulotero','miatt','ubrpagosmex',
        'undostres','rappipro','cashiecommerce','aplazo','aplaz'
    },
    'gasolina': {
        'oxxogas', 'costcogas'
    },
    'tech': {
        'apple','microsoft','google','openai','canva'
    },
    'servicios': {
        'servaguadren','urbani','melimas','allianzmexico','bae','parco'
    },
    'otros': {'caliente','nayax'}
}

In [48]:
# invert CATEGORIES so you can quickly go comercio_norm → big‐category
reverse_map = {}
for cat, providers in CATEGORIES.items():
    for prov in providers:
        reverse_map[prov] = cat

# now map each row (fall back to 'otros' if you didn't list it explicitly)
df['big_categoria'] = df['comercio'].map(reverse_map).fillna('otros')


In [49]:
# “How many in each big category?”
print(df['big_categoria'].value_counts())

# A quick peek at any unmapped comercio_norm’s
missing = set(df['comercio']) - set(reverse_map)
print("Unmapped comercios:", missing)


big_categoria
retail_fisico      70700
transporte         70586
retail_online      41901
comida             40163
entretenimiento    35159
finanzas           28490
domiciliado        18197
tech               12230
telecom            11311
servicios           7272
otros               6871
gasolina            3131
Name: count, dtype: int64
Unmapped comercios: set()


In [54]:

print(df)
df.to_csv("cleanedDataTransactions.csv")

                                              id       fecha tipo_venta  \
0       91477f382c3cf63ab5cd9263b502109243741158  2022-01-02    digital   
1       91477f382c3cf63ab5cd9263b502109243741158  2022-01-05    digital   
2       91477f382c3cf63ab5cd9263b502109243741158  2022-01-05    digital   
3       91477f382c3cf63ab5cd9263b502109243741158  2022-01-05    digital   
4       91477f382c3cf63ab5cd9263b502109243741158  2022-01-05    digital   
...                                          ...         ...        ...   
346006  710f13196a0f9109e22bc73c2fe76f22fbef3dc1  2023-01-26     fisica   
346007  710f13196a0f9109e22bc73c2fe76f22fbef3dc1  2023-01-27     fisica   
346008  710f13196a0f9109e22bc73c2fe76f22fbef3dc1  2023-01-29     fisica   
346009  710f13196a0f9109e22bc73c2fe76f22fbef3dc1  2023-01-29    digital   
346010  710f13196a0f9109e22bc73c2fe76f22fbef3dc1  2023-01-30     fisica   

         monto  big_categoria  comercio  
0         5.99  retail_online    amazon  
1        13.01 